## **Merge Union(API + Scraping)**

In [29]:
import requests
import pandas as pd
import numpy as np
import os
import re
import time
from datetime import datetime

# ==============================================================================
# ⚙️ 1. CONFIGURACIÓN Y LLAVES (ETAPA 1)
# ==============================================================================
API_KEY_VC = "a7117854a0msh861c287680abd70p1dfc59jsnfffd314081ec" #
ARCHIVO_SENAMHI = "SENAMHI_ESTACIONES_FINAL.csv"

# Red Nacional de Monitoreo (15 Aeropuertos)
AEROPUERTOS = [
    {"ID": "SPJC", "NOM": "LIMA", "ZONA": "COSTA"},
    {"ID": "SPZO", "NOM": "CUSCO", "ZONA": "SIERRA"},
    {"ID": "SPQU", "NOM": "AREQUIPA", "ZONA": "SIERRA"},
    {"ID": "SPQT", "NOM": "IQUITOS", "ZONA": "SELVA"},
    {"ID": "SPCL", "NOM": "PUCALLPA", "ZONA": "SELVA"},
    {"ID": "SPST", "NOM": "TARAPOTO", "ZONA": "SELVA"},
    {"ID": "SPUR", "NOM": "PIURA", "ZONA": "COSTA"},
    {"ID": "SPHI", "NOM": "CHICLAYO", "ZONA": "COSTA"},
    {"ID": "SPRU", "NOM": "TRUJILLO", "ZONA": "COSTA"},
    {"ID": "SPJB", "NOM": "CAJAMARCA", "ZONA": "SIERRA"},
    {"ID": "SPDB", "NOM": "PTO MALDONADO", "ZONA": "SELVA"},
    {"ID": "SPJI", "NOM": "JULIACA", "ZONA": "SIERRA"},
    {"ID": "SPAY", "NOM": "AYACUCHO", "ZONA": "SIERRA"},
    {"ID": "SPTZ", "NOM": "TUMBES", "ZONA": "COSTA"},
    {"ID": "SPTN", "NOM": "TACNA", "ZONA": "COSTA"}
]

# ==============================================================================
# 🛠️ 2. MÓDULO DE DATOS (ETAPAS 1, 2 Y 3)
# ==============================================================================

def asegurar_base_senamhi():
    """Genera la base terrestre si no existe mediante Scraping."""
    if os.path.exists(ARCHIVO_SENAMHI): return
    print("📡 Scrapeando estaciones SENAMHI...")
    url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/"
    try:
        r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=15)
        regex = r'"nom"\s*:\s*"(.*?)".*?"lat"\s*:\s*(-?\d+\.?\d*).*?"lon"\s*:\s*(-?\d+\.?\d*)'
        matches = re.findall(regex, r.text)
        datos = [{"ESTACION": m[0], "LATITUD": float(m[1]), "LONGITUD": float(m[2])} for m in matches]
        pd.DataFrame(datos).to_csv(ARCHIVO_SENAMHI, index=False)
    except: pass

def etapa_1_clima_y_pronostico(icao):
    """ETAPA 1: Clima actual y predicción horaria."""
    url = "https://visual-crossing-weather.p.rapidapi.com/forecast"
    try:
        params = {"location": f"{icao}, Peru", "aggregateHours": "1", "unitGroup": "metric", "contentType": "json"}
        r = requests.get(url, headers={"x-rapidapi-key": API_KEY_VC, "x-rapidapi-host": "visual-crossing-weather.p.rapidapi.com"}, params=params)
        data = r.json()
        loc = data['locations'][list(data['locations'].keys())[0]]
        actual = loc['currentConditions']
        # Predicción: Siguiente entrada en el historial de pronóstico
        futuro = loc['values'][1] if 'values' in loc else {}
        return {
            "LAT": loc['latitude'], "LON": loc['longitude'],
            "TEMP": actual.get('temp', 0), "VIENTO": actual.get('wspd', 0),
            "HUMEDAD": actual.get('humidity', 0),
            "ESTADO": actual.get('conditions', "N/D"),
            "PREDICCION": futuro.get('conditions', "Estable")
        }
    except: return None

def etapa_2_radar_aereo(lat, lon):
    """ETAPA 2: Escaneo de aeronaves en radio de 90km."""
    try:
        box = {"lamin": lat-0.4, "lamax": lat+0.4, "lomin": lon-0.4, "lomax": lon+0.4}
        r = requests.get("https://opensky-network.org/api/states/all", params=box, timeout=5)
        vuelos = r.json().get("states", [])
        return len(vuelos) if vuelos else 0
    except: return 0

def etapa_3_senamhi_val(lat, lon):
    """ETAPA 3: Validación con estación física."""
    try:
        df = pd.read_csv(ARCHIVO_SENAMHI)
        df['d'] = np.sqrt((df['LATITUD'] - lat)**2 + (df['LONGITUD'] - lon)**2)
        return df.sort_values('d').iloc[0]['ESTACION']
    except: return "N/D"

# ==============================================================================
# 🚀 3. MOTOR DE EJECUCIÓN (DASHBOARD)
# ==============================================================================

def generar_dashboard_nacional():
    asegurar_base_senamhi()
    print(f"--- 🛫 INICIANDO ESCANEO NACIONAL - {datetime.now().strftime('%H:%M:%S')} ---")
    
    data_final = []
    for aero in AEROPUERTOS:
        clima = etapa_1_clima_y_pronostico(aero['ID'])
        if not clima: continue
        
        vuelos = etapa_2_radar_aereo(clima['LAT'], clima['LON'])
        est_tierra = etapa_3_senamhi_val(clima['LAT'], clima['LON'])
        
        # Lógica de Riesgo Corregida
        riesgo = "🟢 BAJO"
        if (clima['HUMEDAD'] or 0) > 92 and aero['ZONA'] == "COSTA": riesgo = "🔴 ALTO"
        elif (clima['VIENTO'] or 0) > 40: riesgo = "🔴 ALTO"
        elif "Rain" in clima['PREDICCION']: riesgo = "🟡 MEDIO"

        data_final.append({
            "HORA": datetime.now().strftime("%H:%M:%S"),
            "COD": aero['ID'],
            "CIUDAD": aero['NOM'],
            "ESTADO_ACTUAL": clima['ESTADO'],
            "PRONOSTICO": clima['PREDICCION'], # <--- PREDICCIÓN (Tipo de clima)
            "TEMP": f"{clima['TEMP']}°C",
            "AVIONES": vuelos,
            "ST_SENAMHI": est_tierra,
            "RIESGO": riesgo
        })
        time.sleep(0.5)

    df = pd.DataFrame(data_final)
    print("\n" + "="*140)
    print(df.to_string(index=False))
    print("="*140)
    return df

if __name__ == "__main__":
    df_maestro = generar_dashboard_nacional()

--- 🛫 INICIANDO ESCANEO NACIONAL - 17:42:04 ---

  HORA   COD      CIUDAD    ESTADO_ACTUAL    PRONOSTICO     TEMP   AVIONES   ST_SENAMHI    RIESGO
17:42:06 SPJC          LIMA      N/D                 Clear    0°C    0     CAMPO DE MARTE  🟢 BAJO
17:42:08 SPZO         CUSCO      N/D                 Clear    0°C    0     CAMPO DE MARTE  🟢 BAJO
17:42:11 SPQU      AREQUIPA      N/D              Overcast 17.8°C    2           EL SALTO  🟢 BAJO
17:42:13 SPQT       IQUITOS      N/D                 Clear    0°C    0     CAMPO DE MARTE  🟢 BAJO
17:42:15 SPCL      PUCALLPA      N/D        Rain, Overcast -1.2°C    2     SANTA CLOTILDE 🟡 MEDIO
17:42:17 SPST      TARAPOTO      N/D              Overcast  7.4°C    4           EL SALTO  🟢 BAJO
17:42:20 SPUR         PIURA      N/D      Partially cloudy -7.9°C    0           EL SALTO  🟢 BAJO
17:42:22 SPHI      CHICLAYO      N/D              Overcast  7.4°C    5           EL SALTO  🔴 ALTO
17:42:24 SPRU      TRUJILLO      N/D              Overcast 17.8°C    

## **Código para generar Dashboard**

In [32]:
# 1. Ejecutamos el motor que integra las 3 etapas (Clima, Radar y SENAMHI)
df_maestro = generar_dashboard_nacional() 

# 2. Guardamos físicamente el reporte de inteligencia en tu computadora
df_maestro.to_csv("DASHBOARD_NACIONAL_PERU_DETALLADO.csv", index=False) 

# 3. Visualizamos el DataFrame aquí mismo para auditoría inmediata
print("=== REPORTE DE INTELIGENCIA OPERATIVA (15 PROVINCIAS) ===")
display(df_maestro)

--- 🛫 INICIANDO ESCANEO NACIONAL - 17:51:35 ---

  HORA   COD      CIUDAD    ESTADO_ACTUAL    PRONOSTICO     TEMP   AVIONES   ST_SENAMHI    RIESGO
17:51:37 SPJC          LIMA      N/D                 Clear    0°C    0     CAMPO DE MARTE  🟢 BAJO
17:51:39 SPZO         CUSCO      N/D                 Clear    0°C    0     CAMPO DE MARTE  🟢 BAJO
17:51:41 SPQU      AREQUIPA      N/D              Overcast 17.8°C    0           EL SALTO  🟢 BAJO
17:51:44 SPQT       IQUITOS      N/D                 Clear    0°C    0     CAMPO DE MARTE  🟢 BAJO
17:51:46 SPCL      PUCALLPA      N/D        Rain, Overcast -1.2°C    5     SANTA CLOTILDE 🟡 MEDIO
17:51:48 SPST      TARAPOTO      N/D              Overcast  7.4°C    5           EL SALTO  🟢 BAJO
17:51:50 SPUR         PIURA      N/D      Partially cloudy -7.9°C    3           EL SALTO  🟢 BAJO
17:51:53 SPHI      CHICLAYO      N/D              Overcast  7.4°C    5           EL SALTO  🔴 ALTO
17:51:55 SPRU      TRUJILLO      N/D              Overcast 17.8°C    

,HORA,COD,CIUDAD,ESTADO_ACTUAL,PRONOSTICO,TEMP,AVIONES,ST_SENAMHI,RIESGO
0,17:51:37,SPJC,LIMA,N/D,Clear,0°C,0,CAMPO DE MARTE,🟢 BAJO
1,17:51:39,SPZO,CUSCO,N/D,Clear,0°C,0,CAMPO DE MARTE,🟢 BAJO
2,17:51:41,SPQU,AREQUIPA,N/D,Overcast,17.8°C,0,EL SALTO,🟢 BAJO
3,17:51:44,SPQT,IQUITOS,N/D,Clear,0°C,0,CAMPO DE MARTE,🟢 BAJO
4,17:51:46,SPCL,PUCALLPA,N/D,"Rain, Overcast",-1.2°C,5,SANTA CLOTILDE,🟡 MEDIO
5,17:51:48,SPST,TARAPOTO,N/D,Overcast,7.4°C,5,EL SALTO,🟢 BAJO
6,17:51:50,SPUR,PIURA,N/D,Partially cloudy,-7.9°C,3,EL SALTO,🟢 BAJO
7,17:51:53,SPHI,CHICLAYO,N/D,Overcast,7.4°C,5,EL SALTO,🔴 ALTO
8,17:51:55,SPRU,TRUJILLO,N/D,Overcast,17.8°C,0,EL SALTO,🟢 BAJO
9,17:51:57,SPJB,CAJAMARCA,N/D,Clear,0°C,0,CAMPO DE MARTE,🟢 BAJO
